**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [1]:
from tensorflow.config import list_physical_devices

list_physical_devices('GPU')

2025-08-25 02:25:49.629636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756088749.647903    2408 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756088749.654321    2408 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756088749.671172    2408 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756088749.671193    2408 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756088749.671195    2408 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 텐서플로 모델 서빙하기

## 텐서플로 서빙 사용

가장 먼저 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보낸다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [2]:
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.backend import clear_session
from tensorflow.keras import Sequential, Input
from tensorflow import uint8
from tensorflow.keras.layers import Flatten, Rescaling, Dense
from pathlib import Path

mnist = load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
clear_session()
model = Sequential([
    Input((28, 28), dtype=uint8),
    Flatten(),
    Rescaling(1 / 255),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model_name = 'my_mnist_model'
model_version = '0001'
model_path = Path(model_name) / model_version
model.export(model_path)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


I0000 00:00:1756088754.042803    2408 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5267 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1756088756.098681    2481 service.cc:152] XLA service 0x7743f0004b00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756088756.098729    2481 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-08-25 02:25:56.122538: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1756088756.167939    2481 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-25 02:25:56.883981: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36_0', 92 bytes spill stores, 96 bytes spill loads

2025-08-25 02:25:57.130771: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36

  76/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2264 - loss: 2.2108

...

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7165 - loss: 1.0678

2025-08-25 02:26:07.974748: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-25 02:26:07.977453: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 296 bytes spill stores, 296 bytes spill loads

2025-08-25 02:26:08.145081: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-25 02:26:08.201028: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 792 bytes spill stores, 844 bytes spill loads

2025-08-25 02:26:08.288682: I external/local_xla

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.7165 - loss: 1.0675 - val_accuracy: 0.9046 - val_loss: 0.3699
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8973 - loss: 0.3698 - val_accuracy: 0.9194 - val_loss: 0.2998
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9124 - loss: 0.3087 - val_accuracy: 0.9276 - val_loss: 0.2678
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9220 - loss: 0.2792 - val_accuracy: 0.9346 - val_loss: 0.2443
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9277 - loss: 0.2562 - val_accuracy: 0.9396 - val_loss: 0.2274
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9342 - loss: 0.2361 - val_accuracy: 0.9408 - val_loss: 0.2119
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9380 - loss: 0.2172 - val_accuracy: 0.9452 - val_loss: 0.1988
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9423 - loss: 0.2067 - val_accura

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Saved artifact at 'my_mnist_model/0001'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  131137331492240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131137331492048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131137331487824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131137331493008: TensorSpec(shape=(), dtype=tf.resource, name=None)


파일 트리를 살핀다:

In [3]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사한다:

In [4]:
!saved_model_cli show --dir '{model_path}'

2025-08-25 02:26:47.101101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756088807.119954    3300 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756088807.126243    3300 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756088807.145934    3300 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756088807.145978    3300 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756088807.145984    3300 computation_placer.cc:177] computation placer alr

In [5]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2025-08-25 02:26:50.021521: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756088810.034681    3317 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756088810.038833    3317 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756088810.051383    3317 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756088810.051432    3317 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756088810.051439    3317 computation_placer.cc:177] computation placer alr

In [6]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2025-08-25 02:26:52.633452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756088812.648809    3333 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756088812.652884    3333 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756088812.664724    3333 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756088812.664779    3333 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756088812.664785    3333 computation_placer.cc:177] computation placer alr

더 자세한 내용을 보려면 다음 명령을 실행한다:
```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행한다.
```bash
docker pull tensorflow/serving:latest-gpu

docker run -e MODEL_NAME=my_mnist_model --gpus all -it -p 8500:8500 -p 8501:8501 -v path\to\my_mnist_model:/models/my_mnist_model tensorflow/serving:latest-gpu
```

### REST API로 TF 서빙에 쿼리하기

다음으로 TF 서빙에 REST 쿼리를 전송한다:

In [7]:
from json import dumps

X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정한다.
request_json = dumps({'signature_name': 'serving_default', 'instances': X_new.tolist()})

In [8]:
request_json[:100] + '...' + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

이제 텐서플로 서빙의 REST API를 사용하여 예측한다:

In [9]:
from requests import post

server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = post(server_url, request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [10]:
from numpy import array

y_proba = array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

### gRPC API로 TF 서빙에 쿼리하기

In [12]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest
from tensorflow import make_tensor_proto

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
input_name = 'keras_tensor'
request.inputs[input_name].CopyFrom(make_tensor_proto(X_new))

In [13]:
from grpc import insecure_channel
from tensorflow_serving.apis.prediction_service_pb2_grpc import PredictionServiceStub

channel = insecure_channel('172.17.0.3:8500')
predict_service = PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

응답을 텐서로 변환한다:

In [14]:
from tensorflow import make_ndarray

output_name = 'output_0'
outputs_proto = response.outputs[output_name]
y_proba = make_ndarray(outputs_proto)

In [15]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

### 새 모델 버전 배포

In [16]:
# 새로운 MNIST 모델 버전 빌드 및 훈련
model = Sequential([
    Input((28, 28), dtype=uint8),
    Flatten(),
    Rescaling(1 / 255),
    Dense(50, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10


2025-08-25 02:30:22.820845: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-08-25 02:30:23.248747: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 940 bytes spill stores, 940 bytes spill loads

2025-08-25 02:30:23.255978: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-08-25 02:30:23.404178: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 1180 bytes spill stores, 1240 bytes spill loads

2025-08-25 02:30:23.418436: I external

1709/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6739 - loss: 1.1744

2025-08-25 02:30:29.301921: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-08-25 02:30:29.685202: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-08-25 02:30:29.751221: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 988 bytes spill stores, 988 bytes spill loads

2025-08-25 02:30:29.825790: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 956 bytes spill stores, 956 bytes spill loads

2025-08-25 02:30:30.008837: I external/l

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6747 - loss: 1.1716

2025-08-25 02:30:33.319179: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-08-25 02:30:33.365708: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 348 bytes spill stores, 348 bytes spill loads

2025-08-25 02:30:33.429757: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-08-25 02:30:33.567267: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 816 bytes spill stores, 816 bytes spill loads



1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.6748 - loss: 1.1714 - val_accuracy: 0.9036 - val_loss: 0.3462
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9010 - loss: 0.3430 - val_accuracy: 0.9232 - val_loss: 0.2776
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9179 - loss: 0.2880 - val_accuracy: 0.9292 - val_loss: 0.2434
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9279 - loss: 0.2484 - val_accuracy: 0.9362 - val_loss: 0.2241
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9355 - loss: 0.2263 - val_accuracy: 0.9416 - val_loss: 0.2043
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9417 - loss: 0.2027 - val_accuracy: 0.9460 - val_loss: 0.1883
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9473 - loss: 0.1840 - val_accuracy: 0.9500 - val_loss: 0.1740
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9501 - loss: 0.1753 - val_accura

In [17]:
model_version = '0002'
model_path = Path(model_name) / model_version
model.export(model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Saved artifact at 'my_mnist_model/0002'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  131139181529424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181530000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181528272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181531728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181529232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181532880: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [18]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**경고**: 텐서플로 서빙이 새 모델을 로드하기까지 잠시 기다려야 할 수 있다.

In [19]:
server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = post(server_url, request_json)
response.raise_for_status()
response = response.json()

In [20]:
response.keys()

dict_keys(['predictions'])

In [21]:
y_proba = array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

# 모바일 또는 임베디드 디바이스에 모델 배포하기

In [22]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open('my_converted_savedmodel.tflite', 'wb') as f:
    f.write(tflite_model)

W0000 00:00:1756089074.330784    2408 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1756089074.330825    2408 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-25 02:31:14.334485: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: my_mnist_model/0002
2025-08-25 02:31:14.337918: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-25 02:31:14.337929: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: my_mnist_model/0002
I0000 00:00:1756089074.342882    2408 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-08-25 02:31:14.343514: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-25 02:31:14.371016: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: my_mnist_model/0002
2025-08-25 02:31:14.378172: I tensorflow/cc/saved_model/loader.cc:471] Sa

In [23]:
# 케라스 모델을 변환한다.
converter = lite.TFLiteConverter.from_keras_model(model)

In [24]:
converter.optimizations = [lite.Optimize.DEFAULT]

In [25]:
tflite_model = converter.convert()
with open('my_converted_keras_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp8fhqa0y3/assets


INFO:tensorflow:Assets written to: /tmp/tmp8fhqa0y3/assets


Saved artifact at '/tmp/tmp8fhqa0y3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  131139181529424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181530000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181528272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181531728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181529232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  131139181532880: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1756089074.641045    2408 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1756089074.641090    2408 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-25 02:31:14.641233: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp8fhqa0y3
2025-08-25 02:31:14.641639: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-25 02:31:14.641646: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp8fhqa0y3
2025-08-25 02:31:14.644466: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-25 02:31:14.660894: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp8fhqa0y3
2025-08-25 02:31:14.666641: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 25410 microseconds.


# GPU를 사용하여 계산 속도 향상하기

텐서플로우가 GPU를 볼 수 있는지 확인한다:

In [26]:
physical_gpus = list_physical_devices('GPU')
physical_gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

텐서플로 스크립트에서 GPU \#0과 \#1(PCI 순서 기준)만 사용하려면 스크립트를 시작하기 전에 환경 변수 `CUDA_DEVICE_ORDER=PCI_BUS_ID`와 `CUDA_VISIBLE_DEVICES=0,1`을 설정한다. 또는 스크립트 자체에서 텐서플로를 사용하기 전에 지정한다.

## GPU RAM 관리

RAM 용량을 GPU당 2GB로 제한하려면:

In [27]:
# for gpu in physical_gpus:
#     config.set_logical_device_configuration(gpu, [config.LogicalDeviceConfiguration(2048)])

점진적으로 텐서플로가 메모리를 점유하도록 하려면(프로세스가 종료될 때만 메모리를 해제한다):

In [28]:
from tensorflow.config.experimental import set_memory_growth

# for gpu in physical_gpus:
#     set_memory_growth(gpu, True)

이와 동일하게, 텐서플로를 사용하기 전에 `TF_FORCE_GPU_ALLOW_GROWTH` 환경 변수를 `true`로 설정할 수 있다.

물리적 GPU를 두 개의 논리적 GPU로 분할한다:

In [29]:
from tensorflow.config import LogicalDeviceConfiguration

# set_logical_device_configuration(physical_gpus[0], [LogicalDeviceConfiguration(2048), LogicalDeviceConfiguration(2048)])

In [30]:
from tensorflow.config import list_logical_devices

logical_gpus = list_logical_devices('GPU')
logical_gpus

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

## 디바이스에 연산 및 변수 배치하기

모든 변수 및 연산 배치를 기록하려면(이 작업은 텐서플로를 임포팅한 직후에 실행해야 한다):

In [31]:
from tensorflow import get_logger
from tensorflow.debugging import set_log_device_placement

get_logger().setLevel("DEBUG")  # 디폴트 로그 수준은 INFO다.
set_log_device_placement(True)

In [32]:
from tensorflow import Variable

a = Variable([1., 2., 3.])  # float32 변수는 GPU로 이동한다.
a.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [33]:
b = Variable([1, 2, 3])  # int32 변수는 CPU로 이동한다.
b.device

'/job:localhost/replica:0/task:0/device:CPU:0'

`tf.device()` 컨텍스트를 사용하여 원하는 장치에 변수 및 연산을 수동으로 배치할 수 있다:

In [34]:
from tensorflow import device

with device('/CPU:0'):
    c = Variable([1., 2., 3.])
c.device

'/job:localhost/replica:0/task:0/device:CPU:0'

존재하지 않거나 커널이 없는 장치를 지정하면 텐서플로는 자동으로 기본 장치를 사용한다:

In [35]:
with device('/GPU:1234'):
    d = Variable([1., 2., 3.])
d.device

'/job:localhost/replica:0/task:0/device:GPU:0'

텐서플로에서 존재하지 않는 장치를 사용하려고 할 때 기본 장치로 되돌아가지 않고 예외를 발생시키려면:

In [36]:
from tensorflow.config import set_soft_device_placement
from tensorflow.errors import InvalidArgumentError

set_soft_device_placement(False)
try:
    with device('/GPU:1000'):
        d = Variable([1., 2., 3.])
except InvalidArgumentError as ex:
    print(ex)
set_soft_device_placement(True)  # 소프트 배치로 돌아가기

Could not satisfy device specification '/job:localhost/replica:0/task:0/device:GPU:1000'. enable_soft_placement=0. Supported device types [GPU, CPU]. All available devices [/job:localhost/replica:0/task:0/device:GPU:0, /job:localhost/replica:0/task:0/device:CPU:0].


## 여러 디바이스에서 병렬 실행

inter-op 또는 intra-op 스레드 수를 설정하려는 경우(CPU 포화를 방지하거나 완벽하게 재현 가능한 테스트 케이스를 실행하기 위해 텐서플로를 단일 스레드로 만들고자 하는 경우 유용하다):

In [37]:
from tensorflow.config.threading import set_inter_op_parallelism_threads

# set_inter_op_parallelism_threads(10)
# set_intra_op_parallelism_threads(10)

# 여러 디바이스에서 모델 훈련하기

## 분산 전략 API를 사용한 대규모 훈련

In [38]:
from tensorflow.keras.layers import Reshape, Conv2D, MaxPool2D, Dropout


# 케라스를 사용하여 MNIST용 CNN 모델을 생성한다.
def create_model():
    return Sequential([
        Input((28, 28), dtype=uint8),
        Reshape([28, 28, 1]),
        Rescaling(1 / 255),
        Conv2D(64, 7, padding='same', activation='relu'),
        MaxPool2D(2),
        Conv2D(128, 3, padding='same', activation='relu'),
        Conv2D(128, 3, padding='same', activation='relu'),
        MaxPool2D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(.5),
        Dense(10, activation='softmax')
    ])

In [39]:
from tensorflow import distribute

strategy = distribute.MirroredStrategy()
with strategy.scope():
    model = create_model()  # 일반적인 케라스 모델을 생성한다.
    model.compile('SGD', 'sparse_categorical_crossentropy', metrics=['accuracy'])  # 모델을 컴파일한다.
batch_size = 100  # 복제본 수로 나누어지는 것이 바람직하다.
model.fit(X_train, y_train, batch_size, 10, validation_data=(X_valid, y_valid))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Epoch 1/10
547/550 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3633 - loss: 1.9373

2025-08-25 02:31:21.052198: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
2025-08-25 02:31:21.052294: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


550/550 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.3648 - loss: 1.9334 - val_accuracy: 0.8954 - val_loss: 0.3626
Epoch 2/10
  8/550 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7991 - loss: 0.6098  

...

550/550 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9018 - loss: 0.3354 - val_accuracy: 0.9626 - val_loss: 0.1352
Epoch 4/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9269 - loss: 0.2509 - val_accuracy: 0.9706 - val_loss: 0.1035
Epoch 5/10
  9/550 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9223 - loss: 0.2672  

2025-08-25 02:31:34.833799: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


550/550 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9396 - loss: 0.2090 - val_accuracy: 0.9740 - val_loss: 0.0897
Epoch 6/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9491 - loss: 0.1800 - val_accuracy: 0.9762 - val_loss: 0.0802
Epoch 7/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9530 - loss: 0.1606 - val_accuracy: 0.9788 - val_loss: 0.0735
Epoch 8/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9572 - loss: 0.1480 - val_accuracy: 0.9790 - val_loss: 0.0692
Epoch 9/10
  9/550 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9616 - loss: 0.1247  

2025-08-25 02:31:52.450521: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


550/550 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9609 - loss: 0.1357 - val_accuracy: 0.9816 - val_loss: 0.0638
Epoch 10/10
550/550 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9642 - loss: 0.1217 - val_accuracy: 0.9816 - val_loss: 0.0603


In [40]:
model.distribute_strategy

In [41]:
model.predict(X_new).round(2)  # 배치가 모든 복제본에 분할된다.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step


array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [42]:
from tensorflow.keras.models import load_model

# 모델을 저장해도 분산 전략이 보존되지 않는다.
model.save('my_mirrored_model.keras')
model = load_model('my_mirrored_model.keras')
model.distribute_strategy

In [43]:
with strategy.scope():
    model = load_model('my_mirrored_model.keras')

In [44]:
model.distribute_strategy

사용할 GPU 리스트를 지정하려는 경우:

In [45]:
strategy = distribute.MirroredStrategy(['/GPU:0', '/GPU:1'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


기본 all-reduce 알고리즘을 변경하려는 경우:

In [46]:
strategy = distribute.MirroredStrategy(cross_device_ops=distribute.HierarchicalCopyAllReduce())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


`CentralStorageStrategy`을 사용하려는 경우:

In [47]:
strategy = distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:GPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:GPU:0'


INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:GPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:GPU:0'


In [48]:
distribute.cluster_resolver.TPUClusterResolver

tensorflow.python.distribute.cluster_resolver.tpu.tpu_cluster_resolver.TPUClusterResolver

In [49]:
from sys import modules
from os import environ
from tensorflow.config import experimental_connect_to_cluster
from tensorflow.tpu.experimental import initialize_tpu_system

# Google Colab에서 TPU로 훈련하기:
if 'google.colab' in modules and 'COLAB_TPU_ADDR' in environ:
    tpu_address = 'grpc://' + environ['COLAB_TPU_ADDR']
else:
    tpu_address = ''
# resolver = distribute.cluster_resolver.TPUClusterResolver(tpu_address)
# experimental_connect_to_cluster(resolver)
# initialize_tpu_system(resolver)
# strategy = distribute.TPUStrategy(resolver)

## 텐서플로 클러스터에서 모델 훈련하기

텐서플로 클러스터는 일반적으로 서로 다른 컴퓨터에서 병렬로 실행되며 신경망 훈련이나 실행과 같은 작업을 완료하기 위해 서로 대화하는 텐서플로 프로세스의 그룹이다. 클러스터의 각 TF 프로세스를 "태스크"(또는 "TF 서버")라고 한다. IP 주소, 포트, 타입(역할 또는 작업이라고도 함)이 있다. 타입은 `"worker"`, `"chief"`, `"ps"`(파라미터 서버) 또는 `"evaluator"`가 될 수 있다:
* **워커**는 일반적으로 하나 이상의 GPU가 있는 컴퓨터에서 계산을 수행한다.
* **치프**는 계산도 수행하지만, 텐서보드 로그 작성이나 체크포인트 저장과 같은 추가 작업도 처리한다. 클러스터에는 하나의 치프가 있다. 정의되지 않은 경우 워커 #0이 치프가 된다.
* **파라미터 서버**(ps)는 변수 값만 추적하며, 일반적으로 CPU 전용 머신에 있다.
* **이밸류에이터**는 당연히 평가를 담당한다. 일반적으로 클러스터에는 하나의 이밸류에이터가 있다.

같은 유형의 태스크 집합을 흔히 "작업(job)"이라고 한다. 예를 들어 "워커" 작업은 모든 워커 태스크의 집합이다.

텐서플로 클러스터를 시작하려면 먼저 클러스터를 정의해야 한다. 즉, 모든 작업(IP 주소, TCP 포트 및 타입)을 지정해야 한다. 예를 들어, 다음 클러스터 사양은 3개의 태스크(워커 2개, 파라미터 서버 1개)가 있는 클러스터를 정의한다. 작업당 하나의 키가 있는 사전이며, 값은 태스크 주소 목록이다:

In [50]:
cluster_spec = {
    'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222'], 'ps': ['machine-a.example.com:2221']
    # [/job:worker/task:0, /job:worker/task:1], [/job:ps/task:0]
}

In [51]:
environ['TF_CONFIG'] = dumps({'cluster': cluster_spec, 'task': {'type': 'worker', 'index': 0}})

일부 플랫폼(예: 구글 버텍스 AI)에서는 이 환경 변수를 자동으로 설정한다.

텐서플로의 `TFConfigClusterResolver` 클래스는 이 환경 변수에서 클러스터 구성을 읽는다:

In [52]:
resolver = distribute.cluster_resolver.TFConfigClusterResolver()
resolver.cluster_spec()

ClusterSpec({'ps': ['machine-a.example.com:2221'], 'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222']})

In [53]:
resolver.task_type

'worker'

In [54]:
resolver.task_id

0

이제 로컬 컴퓨터에서 두 개의 워커 태스크를 가진 간단한 클러스터를 실행한다. `MultiWorkerMirroredStrategy`을 사용하여 두 태스크로 모델을 훈련한다.

첫 번째 단계는 훈련 코드를 작성하는 것이다. 이 코드를 사용해 두 워커에서 각각 고유한 프로세스로 실행하므로 별도의 파이썬 파일인 `my_mnist_multiworker_task.py`에 작성한다. 코드는 비교적 간단하지만 주의해야 할 몇 가지 중요한 사항이 있다:
* 텐서플로로 다른 작업을 수행하기 전에 `MultiWorkerMirroredStrategy`을 생성한다.
* 워커 중 하나만 텐서보드 로깅을 처리한다. 앞서 언급했듯이 이 작업자를 *치프*라고 한다. 명시적으로 정의되지 않은 경우 워커 #0이 치프다.

In [17]:
%%writefile my_mnist_multiworker_task.py
from tensorflow import distribute
from tf_keras.datasets.mnist import load_data
from tf_keras import Sequential, Input
from tensorflow import uint8
from tf_keras.layers import Reshape, Rescaling, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tempfile import mkdtemp
from tensorflow.io.gfile import rmtree

strategy = distribute.MultiWorkerMirroredStrategy()  # 시작 부분에!
resolver = distribute.cluster_resolver.TFConfigClusterResolver()
print(f'Starting task {resolver.task_type} #{resolver.task_id}')
# MNIST 데이터셋 로드 및 분할
mnist = load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
with strategy.scope():
    model = Sequential([
        Input((28, 28), dtype=uint8),
        Reshape((28, 28, 1)),
        Rescaling(1 / 255),
        Conv2D(64, 7, padding='same', activation='relu'),
        MaxPool2D(2),
        Conv2D(128, 3, padding='same', activation='relu'),
        Conv2D(128, 3, padding='same', activation='relu'),
        MaxPool2D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(.5),
        Dense(10, activation='softmax')
    ])
    model.compile('SGD', 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
if resolver.task_id == 0:  # 치프는 모델을 올바른 위치에 저장한다.
    model.export('my_mnist_multiworker_model')
else:
    tmpdir = mkdtemp()  # 다른 워커는 임시 디렉터리에 저장한다.
    model.export(tmpdir)
    rmtree(tmpdir)  # 마지막에 이 디렉터리를 삭제할 수 있다!

Overwriting my_mnist_multiworker_task.py


실제 애플리케이션에서는 일반적으로 머신당 하나의 워커가 있지만, 이 예제에서는 동일한 머신에서 두 워커를 모두 실행하고 있으므로 두 워커 모두 사용 가능한 모든 GPU RAM(이 머신에 GPU가 있는 경우)을 사용하려고 시도하므로 메모리 부족(OOM) 오류가 발생할 수 있다. 이를 방지하기 위해 `CUDA_VISIBLE_DEVICES` 환경 변수를 사용하여 각 워커에 다른 GPU를 할당한다. 또는 `CUDA_VISIBLE_DEVICES`를 빈 문자열로 설정하여 간단히 GPU 지원을 비활성화한다.

In [18]:
%%bash --bg
export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]}, "task": {"type": "worker", "index": 0}}'
python my_mnist_multiworker_task.py &> my_worker_0.log

In [19]:
%%bash --bg
export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]}, "task": {"type": "worker", "index": 1}}'
python my_mnist_multiworker_task.py &> my_worker_1.log

**참고**: `AutoShardPolicy`에 대한 경고가 표시되면 무시해도 무방하다. 자세한 내용은 [TF 이슈 #42146](https://github.com/tensorflow/tensorflow/issues/42146)을 참고한다.

끝났다! 이제 텐서플로 클러스터가 실행 중이지만 별도의 프로세스에서 실행 중이므로 이 노트북에서는 볼 수 없다(하지만 진행 상황은 `my_worker_*.log`에서 확인할 수 있다).